# But : Créer un masque de la ville sans végétation avec l'image infrarouge Pléiade.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
! pip install rasterio  -q -q -q
! pip install geopandas -q -q -q
! pip install matplotlib -q -q -q

In [3]:
!pip install pyarrow
!pip install opencv-python
!pip install os-sys

In [5]:
import sys
sys.path.append('../src')
from satellite_image import SatelliteImage
from utils import *

lancer ca dans le terminal avant d'importer cv2 :
sudo apt-get update
sudo apt-get install ffmpeg libsm6 libxext6 -y
sudo apt-get install libgl1

In [6]:
import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image as im

In [7]:
update_storage_access()

In [8]:
environment = get_environment()

root_path = get_root_path()
bucket = environment["bucket"]
path_s3_cayenne_data = environment["sources"]["PLEIADES"]
path_local_cayenne_data = os.path.join(root_path, environment["local-path"]["PLEIADES"])
bucket = environment["bucket"]

path_s3_pleiades_data = environment["sources"]["PLEIADES"]
path_s3_bdtopo_data = environment["sources"]["BDTOPO"][2022]["guyane"]
path_local_pleiades_data = environment["local-path"]["PLEIADES"]
path_local_bdtopo_data = environment["local-path"]["BDTOPO"][2022]["guyane"]


fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

In [9]:
# DL PLEIADE
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data}",
        lpath=f"../{path_local_pleiades_data}",
        recursive=True)

In [10]:
# DL BDTOPO
fs.download(
        rpath=f"{bucket}/{path_s3_bdtopo_data}",
        lpath=f"../{path_local_bdtopo_data}",
        recursive=True)

## Chargement données pléiades.

In [10]:
filename = '../data/PLEIADES/Cayenne/16bits/ORT_2022072050325085_U22N/ORT_2022072050325085_0353_0545_U22N_16Bits.jp2'
date = datetime.strptime(re.search(r'ORT_(\d{8})', filename).group(1), '%Y%m%d')
date

In [11]:
image = SatelliteImage.from_raster(
        filename,
        date = date, 
        n_bands = 4,
        dep = "973"
    )
image.normalize()

In [13]:
image.plot([0,1,2])
image.plot([3,1,2])

## D'abord : un masque binaire avec en blanc les bâtiments et les routes, en noir le reste.
Pour commencer ceci, on veut avoir l'image avec la bande infrarouge en format png pour pouvoir utiliser le package OpenCv.

In [12]:
#On va d'abord récuperer l'array avec la bande infrarouge 
#faire un array sans la couleur rouge pour l'image en 51e position de la liste d'images
if not image.normalized:
    image.normalize() 
img = image.array

#avoir la bande infrarouge
img = img[[3,1,2],:, :]

#multiplication par 255 et convertion en uint8 pour avoir le bon format
img = (img * 255).astype(np.uint8)

#convertir l'array en image PIL pour ensuite l'avoir en png
img = im.fromarray(img.transpose(1, 2, 0).astype('uint8'), 'RGB')

#enregistrer l'image en tant que fichier PNG pour pouvoir utiliser OpenCV
img.save('im50.png')

Afin de ne pas faire crasher le kernel, nous allons préalablement créer des fonctions d'affichage des images avec OpenCV (la fonction de base cv2.imshow() fait crasher le kernel)

In [13]:
#images en nuances de gris
def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_RGBA2BGRA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_RGB2BGR)

    return plt.imshow(a, **kwargs, cmap= "gray")

#images en couleurs
def cv2_imshow2(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_RGBA2BGRA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_RGB2BGR)

    return plt.imshow(a, **kwargs, cmap= "brg")


In [194]:
#on lit l'image avec OpenCV
img = cv2.imread('im50.png', cv2.IMREAD_COLOR)

In [39]:
cv2_imshow2(img)

In [113]:
img[1125,1325]
mini = min(img[1125,1325, 0],img[1125,1325, 1])
mini
img[1125,1325, 2] - mini

In [152]:
mini = min(img[1125,1325, 0],img[1125,1325, 1])
mini

In [177]:
img[250,550]

In [154]:
img[1125,1325, 2] - mini

## Idée globale : si le pixel est rouge, le changer en noir, le reste en blanc.
Pour cela, il faut fixer des seuils : quels nuances de rouges colorier en noir ou en blanc ?
Ici les pixels sont en décimal BGR

* Etape 1 : Pour que le pixel soit une nuance de rouge, il faut d'abord vérifier si les valeurs verte et bleue sont proches.
* Etape 2 : On décide que les nuances de rouge très claires correspondent à des toits de bâtiments.
* Etape 3 : On fixe le seuil de la valeur rouge à 110 et la valeur de rouge avec une autre couleur doit être supérieure à 20 minimum.
* Etape 4 : Si le pixel ne vérifie aucune des étapes précédentes, alors il est colorié automatiquement en blanc.

Pour fixer ce seuil, des tests visuels ont été effectués et on a utilisé ce site https://htmlcolorcodes.com/fr/selecteur-de-couleur/ pour comprendre comment fonctionnent les nuances de rouge sur des pixels en décimal (et ce site pour comprendre les pixels en décimal en général http://www.proftnj.com/RGB3.htm).
L'algorithme qui suit met quelques secondes à tourner, il sera long s'il est appliqué sur l'ensemble des images de la base de données.
Pour mieux réussir à faire ce seuillage en étape 3, on devrait créer une fonction racine carré et prendre toutes les valeurs au dessus de la courbe.
Cette fonction est difficile à déterminer... Cependant, on peut la faire uniquement avec deux paramètres : la valeur du rouge et la valeur minimale entre le bleu et le vert. 

In [195]:
# On va parcourir tous les pixels de l'image
for row in range(img.shape[0]):
    for col in range(img.shape[1]):
        b = img[row,col, 0]
        g = img[row,col, 1]
        r = img[row,col, 2]
        mini = min(b,g)
        maxi = max(b,g)
        
        if maxi-mini <= 20 : #étape 1
            
            if r > 200 and mini >= 110 and r>= (20+mini): #étape 2
                img[row, col] = [255, 255, 255] # blanc
            elif  r>= (20+mini) and r >= 110: #étape 3
                img[row, col] = [0, 0, 0] #noir
            else : #étape 4
                img[row, col] = [255, 255, 255] # blanc

        else : #étape 4
            img[row, col] = [255, 255, 255] # blanc

In [196]:
cv2_imshow2(img)

## Maintenant, il serait intéressant de comparer deux images où on sait qu'il y a eu une création de bâtiment.

essai au cas où : détections de contours

In [109]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [110]:
# Grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
# Find Canny edges
edged = cv2.Canny(gray, 100, 200)
  
# Finding Contours
# Use a copy of the image e.g. edged.copy()
# since findContours alters the image
contours, hierarchy = cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    
img_contours = cv2.drawContours(image_test, contours, -1, (0, 255, 0), 5)
cv2_imshow2(img_contours)